In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

import sys, os, time, collections

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'cells'))
# sys.path.append(os.path.join(os.environ['REPO_DIR'], 'annotation'))
# sys.path.append(os.path.join(os.environ['REPO_DIR']bb, 'learning'))

import numpy as np
import scipy
import skimage
from skimage.feature import peak_local_max
from skimage.morphology import watershed
from skimage.measure import regionprops, label, find_contours
from skimage.color import rgb2hsv
from scipy import ndimage as ndi
import pandas
from scipy.signal import argrelmax
from scipy.stats import linregress

from annotation_utilities import *
from registration_utilities import *
from learning_utilities import *
from data_manager import *
from utilities2015 import *
from cell_utilities import *
from metadata import *
from distributed_utilities import download_from_s3

save_folder_path = '/shared/MouseBrainAtlasXiang/XJ/Output/Collect_typical_blobs/'
from xj_utilities import *
fun_create_folder(save_folder_path=save_folder_path)

/usr/local/lib/python2.7/dist-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)
No vtk


Setting environment for AWS compute node


Cannot import mxnet.
Folder already exists: /shared/MouseBrainAtlasXiang/XJ/Output/Collect_typical_blobs/

In [2]:
def fun_get_regionprop(skimage_measure_region, prop):
    """skimage_measure_region: regionprops, returned by skimage.measure.regionprps(), of the type skimage.measure.regionprops
        prop: region property, can be built-in properties like 'area', 'moments_hu', eccentricity', or in addition, 'compactness'
    Returns: regionprops[prop] in float/numpy.ndarray/int, depends on the input.  """
    buildinProps = ['coords','centroid','orientation', 'eccentricity','area','orientation','moments_hu','bbox','equivalent_diameter','label','local_centroid','major_axis_length','solidity','minor_axis_length','perimeter','solidity']
    PI = 3.1415926
    if prop in buildinProps:
        return skimage_measure_region[prop]
    elif prop == 'compactness':
        return float(skimage_measure_region['perimeter'])**2 / float(skimage_measure_region['area']) / (4 * PI)

def fun_get_regionprop_list(skimage_measure_region_list, prop):
    regionprop_list = [fun_get_regionprop(record, prop) for record in skimage_measure_region_list]
    return np.array(regionprop_list)

def fun_save_typical_blobs_info(typical_blobs, prop_to_save, stack, section, dataType='typical',dataFolderName=None):
    # Remove duplicated blobs and save tpical blob information.
    tempTBID = 0
    tempBID = -1
    unique_BID = []
    for record in typical_blobs:
        if tempBID != record[1]:
            tempBID = record[1]
            unique_BID.append(tempTBID)
        tempTBID = tempTBID + 1
    unique_BID = np.array(unique_BID)
    typical_blob_regionprops = np.array([record[2] for record in typical_blobs])[unique_BID]
    fun_save_regionprops(regionprop_List=typical_blob_regionprops,prop_to_save=prop_to_save,stack=stack,sec=section,dataType=dataType, dataFolderName=dataFolderName)

def fun_get_matched_blobs_IDs(matched_paris):
    ''' matched_pairs: a list of [section, blobID, typical_blob_props, scan_sec, matched_blob, matched_blob_props, sim_matrix]
        return: 
            matched_blobs_from_section: a list of sections indicates where do the matched blobs come from
            matched_blobs_with_typicalBlob: a list of typical_blob_ID indicates which typical blob is matched'''
    matched_blobs_from_section = []
    matched_blobs_with_typicalBlob = []
    typical_blobs_id = -1
    blobID = -1
    for record in matched_paris:
        if blobID != record[1]:
            blobID = record[1]
            typical_blobs_id +=1
        temp_matched_blobs_section = record[3]
        temp_num_matched_blobs = len(record[4])
        matched_blobs_from_section += [temp_matched_blobs_section] * temp_num_matched_blobs
        matched_blobs_with_typicalBlob += [typical_blobs_id] * temp_num_matched_blobs
    return matched_blobs_from_section, matched_blobs_with_typicalBlob    
    
    
def fun_save_matched_blobs_info(matched_paris, prop_to_save, stack, section, dataType='matched', dataFolderName=None):
    matched_blob_regionprops = [record[5] for record in matched_paris]
    matched_blob_regionprops = np.concatenate(tuple(matched_blob_regionprops)).tolist()
    fun_save_regionprops(regionprop_List=matched_blob_regionprops, prop_to_save=prop_to_save, stack=stack, sec=section, dataType=dataType, dataFolderName=dataFolderName)
    
    matched_blobs_from_section, matched_blobs_with_typicalBlob = fun_get_matched_blobs_IDs(matched_paris=matched_paris)
    matched_blobs_from_section = np.array(matched_blobs_from_section, dtype=np.int16)
    matched_blobs_with_typicalBlob = np.array(matched_blobs_with_typicalBlob, dtype=np.int16)
    bp.pack_ndarray_file(matched_blobs_with_typicalBlob, get_typical_cell_data_filepath(what='TBID', stack=stack, sec=section, dataType=dataType, dataFolderName=dataFolderName) )
    bp.pack_ndarray_file(matched_blobs_from_section, get_typical_cell_data_filepath(what='section', stack=stack, sec=section, dataType=dataType, dataFolderName=dataFolderName) )
   

In [3]:
def fun_save_regionprops(regionprop_List, prop_to_save, stack, sec, dataType='typical', dataFolderName=None):
    for tempProp in prop_to_save:
        tempProp_data = fun_get_regionprop_list(skimage_measure_region_list=regionprop_List, prop=tempProp)
        
        if tempProp == 'coords':
            tempProp_data = map(lambda data: np.array(data, dtype=np.int16), tempProp_data)
        elif tempProp == 'moments_hu':
            tempProp_data = map(lambda data: np.array(data, dtype=np.float32), tempProp_data)
            tempProp_data = np.row_stack(tuple(tempProp_data))
        elif tempProp == 'centroid':
            tempProp_data = map(lambda data: np.array(data, dtype=np.float32), tempProp_data)
            tempProp_data = np.row_stack(tuple(tempProp_data))
        elif tempProp == 'area':
            tempProp_data = np.array(tempProp_data, np.int32)
        elif tempProp == 'eccentricity':
            tempProp_data = np.array(tempProp_data, np.float32)
        elif tempProp == 'equivalent_diameter':
            tempProp_data = np.array(tempProp_data, np.float32)
        elif tempProp == 'compactness':
            tempProp_data = np.array(tempProp_data, np.float32)
        elif tempProp == 'perimeter':
            tempProp_data = np.array(tempProp_data, np.int32)
        elif tempProp == 'solidity':
            tempProp_data = np.array(tempProp_data, np.float32)
        tempFp = get_typical_cell_data_filepath(what=tempProp,stack=stack,sec=sec, dataType=dataType, dataFolderName=dataFolderName)
        create_if_not_exists(os.path.dirname(tempFp))
        if tempFp.endswith('.hdf'):
            save_hdf_v2(tempProp_data, fn=tempFp)
        elif tempFp.endswith('.bp'):
            bp.pack_ndarray_file(tempProp_data, tempFp)
        else:
            print 'Unrecognized data type. Save failed.'
    return 0

In [4]:
scan_parameters = {}

In [5]:
scan_parameters['dataFolderName'] = 'AddFeature_Reduce_FST'
scan_parameters['stack'] = 'MD589'
scan_parameters['patch_size'] = 448
scan_parameters['patch_half_size'] = scan_parameters['patch_size']/2
scan_parameters['stride'] = 112

scan_parameters['section_limits'] = metadata_cache['section_limits'][scan_parameters['stack']]
scan_parameters['o_crop'] = True
scan_parameters['o_clear_border'] = True
scan_parameters['o_relabel'] = True
scan_parameters['oriImL1'], scan_parameters['oriImL0'] = metadata_cache['image_shape'][scan_parameters['stack']]
scan_parameters['scan_section_range'] = 1
scan_parameters['prop'] = ['area', 'eccentricity','centroid']
# scan_parameters['prop'] = ['centroid','eccentricity','area','orientation','moments_hu','bbox','equivalent_diameter','perimeter','compactness','label','major_axis_length','minor_axis_length']
scan_parameters['prop_for_comparison'] = ['area', 'eccentricity']
if 'moments_hu' in scan_parameters['prop_for_comparison']:
    scan_parameters['compare_weight'] = [1 for i in range(6 + len(scan_parameters['prop_for_comparison']))]
else:
    scan_parameters['compare_weight'] = [1 for i in range(len(scan_parameters['prop_for_comparison']))]
scan_parameters['compare_weight'] = np.array(scan_parameters['compare_weight'])/ float(np.sum(scan_parameters['compare_weight']))
scan_parameters['similarity_threshold'] = 0.9
scan_parameters['false_similarity_threshold'] = 0.7
scan_parameters['o_fix_scan_size'] = True
scan_parameters['scan_size'] = 112
scan_parameters['scan_size_coeff'] = 5
scan_parameters['builtInProps'] = ['centroid','orientation', 'eccentricity','area','moments_hu','bbox','equivalent_diameter','label','local_centroid','major_axis_length','minor_axis_length','perimeter','solidity']
scan_parameters['prop_to_save'] = ['moments_hu','centroid','area','eccentricity','equivalent_diameter', 'compactness', 'orientation','solidity']
scan_parameters['scan_outside_margin'] = 3000
scan_parameters['false_match_sample_num'] = 200
stack = scan_parameters['stack']

In [18]:
contour_df = DataManager.load_annotation_v4(stack=scan_parameters['stack'],by_human=True,suffix='contours',timestamp='latest')
contour_df = convert_annotation_v3_original_to_aligned_cropped(contour_df=contour_df,stack=stack)
structure_name = '7N'
vertice_7N_dic = {record['section']:np.array(record['vertices'],np.int) for _, record in contour_df[contour_df['name']==structure_name].iterrows()}
sectionList_7N = np.sort(vertice_7N_dic.keys())

aws s3 cp --recursive "s3://mousebrainatlas-data/CSHL_labelings_v3/MD589" "/shared/CSHL_labelings_v3/MD589" --exclude "*" --include "*contours*"
latest timestamp:  07292017045157


In [6]:
# Whole image setting:
scan_parameters['crop_0_min'] = 0
scan_parameters['crop_1_min'] = 0
scan_parameters['crop_0_max'] = scan_parameters['oriImL0']
scan_parameters['crop_1_max'] = scan_parameters['oriImL1']
scan_parameters['crop_range_mmxx'] = (scan_parameters['crop_0_min'], scan_parameters['crop_1_min'], scan_parameters['crop_0_max'], scan_parameters['crop_1_max'])
scan_parameters['crop_range_mxmx'] = fun_mmxx_to_mxmx(*scan_parameters['crop_range_mmxx'])
scan_parameters['im0max'] = scan_parameters['crop_0_max'] - scan_parameters['crop_0_min']
scan_parameters['im1max'] = scan_parameters['crop_1_max'] - scan_parameters['crop_1_min']

In [7]:
# margin = 10
# bboxs_7N = {sec:fun_polygon_bbox(vertice_7N_dic[sec]) for sec in sectionList_7N}
# bbox_7N = fun_polygons_bbox(bboxs_7N.values(),margin=margin)
# # bbox_7N = bboxs_7N[154]
# scan_parameters['crop_range_mmxx'] = bbox_7N
# scan_parameters['crop_0_min'], scan_parameters['crop_1_min'],scan_parameters['crop_0_max'],scan_parameters['crop_1_max'] = bbox_7N 
# scan_parameters['im0max'] = scan_parameters['crop_0_max'] - scan_parameters['crop_0_min']
# scan_parameters['im1max'] = scan_parameters['crop_1_max'] - scan_parameters['crop_1_min']
# scan_parameters['crop_range_mxmx'] = fun_mmxx_to_mxmx(*scan_parameters['crop_range_mmxx'])

In [7]:
# Get avaliable sections:
stack = scan_parameters['stack']
section_begin, section_end = scan_parameters['section_limits']
# load_section_num = 100
# load_start_section = 152 - section_begin
all_valid_section = fun_get_valid_section_list(stack=stack)
# secList = all_valid_section[load_start_section:load_section_num + load_start_section]
# secList = list(set(all_valid_section).intersection(sectionList_7N))
secList = all_valid_section
scan_parameters['secList'] = secList

In [139]:
# # loading data
# image = {}
# for tempSec in secList:
#     try:
#         image[tempSec] = fun_crop_images(DataManager.load_image_v2(stack=stack, section=tempSec, version='jpeg', prep_id=2),*scan_parameters['crop_range_mmxx'],im0max=scan_parameters['oriImL0'],im1max=scan_parameters['oriImL1'] )
#     except:
#         sys.stderr.write('Cannot import section %d...\n'%tempSec)
#         continue

In [8]:
def fun_collect_blobs(section, scan_parameters, collectFlaseMatched=False, saveMatched=True):
    scan_range = scan_parameters['scan_section_range']
    im0max = scan_parameters['im0max']
    im1max = scan_parameters['im1max']
    prop = scan_parameters['prop']
    prop_for_comparison = scan_parameters['prop_for_comparison']
    compare_weight = scan_parameters['compare_weight']
    o_simil_threshold = scan_parameters['similarity_threshold']
    o_false_simil_threshold = scan_parameters['false_similarity_threshold'] # Option. Default is equal to o_simil_threshold
    o_fix_scan_size = scan_parameters['o_fix_scan_size']
    o_scan_size_coeff = scan_parameters['scan_size_coeff']
    o_scan_size = scan_parameters['scan_size']  
    secList = scan_parameters['secList'] 
    stack = scan_parameters['stack']
    dataFolderName = scan_parameters['dataFolderName']
    # data_typical_blobs = {}
    # data_matched_paris = {}
    # data_false_typical_blobs = {}
    # data_false_matched_pairs = {}
    # data_blob_prop_dic = {}
    # data_blob_idx_selection = {}
#     section = 154
    # for section in secList:
    typical_blobs = []
    matched_paris = []
    false_typical_blobs = []
    false_matched_paris = []
    cell_centroids = {}
    cell_numbers = {}
    cell_global_coord = {}
    im_blob_prop = {}
    im_label = {}
    im_BW = {}
    sec_load_data_list = range(section - scan_range, section + scan_range + 1)
    scan_section = list(sec_load_data_list)
    scan_section.remove(section)
    ### loading data and reconstructed labeled images ###
    tempList = list(sec_load_data_list)
    for tempSec in tempList:
        if tempSec in secList:
            cell_global_coord[tempSec] = load_cell_data('coords', stack=stack, sec=tempSec)
            _, temp_im_blob_prop, _ = fun_reconstruct_labeled_image(cell_global_coord[tempSec],crop_range= scan_parameters['crop_range_mxmx'], 
                                                                        oriImL0=scan_parameters['oriImL0'],oriImL1=scan_parameters['oriImL1'])
#             im_label[tempSec] = temp_im_label
#             im_BW[tempSec] = temp_im_label > 0
            im_blob_prop[tempSec] = np.array(temp_im_blob_prop)
        else:
            sys.stderr.write('Warning: missing section %d'%tempSec)
            scan_section.remove(tempSec)
            sec_load_data_list.remove(tempSec)

    ### Start getting region properties of each blob    ###
    blob_prop_dic = fun_regionprops_dic(im_blob_prop=im_blob_prop,scan_parameters=scan_parameters)
    # data_blob_prop_dic[section] = blob_prop_dic[section] # Save data
    blob_idx_selection = {tempSec : np.logical_and.reduce(np.row_stack(
        (
        #      blob_prop_dic[tempSec]['compactness']<2.5,
         blob_prop_dic[tempSec]['area']<3000,
         blob_prop_dic[tempSec]['area']>200))) for tempSec in sec_load_data_list}  

    # data_blob_idx_selection[section] = blob_idx_selection[section] # Save data

    n_blobs = {tempSec: len(im_blob_prop[tempSec]) for tempSec in im_blob_prop.keys()}
    secList_in_BlobPropDic = im_blob_prop.keys()
    if set(scan_section).issubset(set(secList_in_BlobPropDic)):
        pass
    else:
        print('Warrning: Scaned section(s) not included in input im_blob_prop')



    ### Start scanning ###
    for blobID in range(n_blobs[section]):
        if (blobID % 1000 == 0):
            print('Section %d Finished percentage: %f'%(section, (float(blobID)*100 / n_blobs[section]) ))

        temp_curr_blob_props = {}
        for tempProp in prop:
            temp_curr_blob_props[tempProp] = blob_prop_dic[section][tempProp][blobID]
        if not blob_idx_selection[section][blobID]:
    #         print('Blob %d eliminated'%blobID)
            continue

        tempB1_idx_loc = temp_curr_blob_props['centroid']
        if o_fix_scan_size:
            temp_next_sec_range, local_cloc = fun_scan_range(tempB1_idx_loc,o_scan_size,im0max=im0max,im1max=im1max,o_form='2D')
            temp_next_sec_range_1D,_ = fun_scan_range(tempB1_idx_loc,o_scan_size,im0max=im0max,im1max=im1max)
        else:
            temp_next_sec_range, local_cloc = fun_scan_range(tempB1_idx_loc,o_scan_size_coeff*fun_radius_bbox(*temp_curr_blob_props.bbox),im0max=im0max,im1max=im1max,o_form='2D')
            temp_next_sec_range_1D,_ = fun_scan_range(tempB1_idx_loc,o_scan_size_coeff*fun_radius_bbox(*temp_curr_blob_props.bbox),im0max=im0max,im1max=im1max)


        for tempSec in scan_section:
    #         print('Blbo %d left. Start scanning'%blobID)
            if tempSec not in secList_in_BlobPropDic:
                continue

            # Find blobs at the nearby location in the scaned section
            tempBlobInside = fun_blobs_in_polygen(blob_prop_dic[tempSec]['centroid'],temp_next_sec_range,coor_order='cr')
            tempBlobInsideIndex = np.where(tempBlobInside)[0]
            temp_num_blob = len(tempBlobInsideIndex)
            if collectFlaseMatched == True:
                tempBlobOutside = fun_blobs_out_polygen(blob_prop_dic[tempSec]['centroid'], temp_next_sec_range, coor_order='cr', margin=scan_parameters['scan_outside_margin'])
                tempBlobOutsideIndex = np.where(tempBlobOutside)[0]
                tempBlobOutsideIndex = np.random.choice(tempBlobOutsideIndex, size=scan_parameters['false_match_sample_num'], replace=False)
                temp_num_outblob = len(tempBlobOutsideIndex)

            ### Typical blobs###

            if temp_num_blob:
                temp_sim = {}
                for temp_prop in prop_for_comparison:
                          temp_sim[temp_prop] = np.array(fun_similarity(temp_curr_blob_props[temp_prop], blob_prop_dic[tempSec][temp_prop][tempBlobInside],distance_type=temp_prop))
                temp_sim_matrix = np.column_stack((temp_sim[temp_prop] for temp_prop in prop_for_comparison))

                #### Blob comparison ####
                temp_weighted_sim = np.dot(temp_sim_matrix,compare_weight)
                temp_compare_result = temp_weighted_sim > o_simil_threshold
                if any(temp_compare_result.tolist()):
                    typical_blobs.append([section,blobID, im_blob_prop[section][blobID]])
                    matched_paris.append([section,blobID,
                                          im_blob_prop[section][blobID],
                                          tempSec,
                                          tempBlobInsideIndex[temp_compare_result],
                                          im_blob_prop[tempSec][tempBlobInsideIndex[temp_compare_result]],
                                          temp_sim_matrix[temp_compare_result,:]])
            else:
                pass

            ### False-typical blobs###
            if collectFlaseMatched == True:
                if temp_num_outblob:
                    temp_false_sim = {}
                    for temp_prop in prop_for_comparison:
                              temp_false_sim[temp_prop] = np.array(fun_similarity(temp_curr_blob_props[temp_prop], blob_prop_dic[tempSec][temp_prop][tempBlobOutsideIndex],distance_type=temp_prop))
                    temp_false_sim_matrix = np.column_stack((temp_false_sim[temp_prop] for temp_prop in prop_for_comparison))

                    #### Blob comparison ####
                    temp_weighted_false_sim = np.dot(temp_false_sim_matrix,compare_weight)
                    temp_false_compare_result = temp_weighted_false_sim > o_false_simil_threshold
                    if any(temp_false_compare_result.tolist()):
                        false_typical_blobs.append([section,blobID, im_blob_prop[section][blobID]])
                        false_matched_paris.append([section,blobID,im_blob_prop[section][blobID],
                                                    tempSec, 
                                                    tempBlobOutsideIndex[temp_false_compare_result], 
                                                    im_blob_prop[tempSec][tempBlobOutsideIndex[temp_false_compare_result]],
                                                    temp_false_sim_matrix[temp_false_compare_result,:]])
                else:
                    pass
    tempFp = get_typical_cell_data_filepath(what='scan_parameters',stack=stack,sec=section,dataFolderName=dataFolderName)
    create_if_not_exists(os.path.dirname(tempFp))
    save_pickle(scan_parameters,tempFp)
    fun_save_typical_blobs_info(typical_blobs=typical_blobs, prop_to_save=scan_parameters['prop_to_save'],stack=stack, section=section, dataType='typical',dataFolderName=dataFolderName)
    if saveMatched == True:
        fun_save_matched_blobs_info(matched_paris=matched_paris, prop_to_save=scan_parameters['prop_to_save'],stack=stack, section=section, dataType='matched',dataFolderName=dataFolderName)
    if collectFlaseMatched == True:
        fun_save_typical_blobs_info(typical_blobs=false_typical_blobs, prop_to_save=scan_parameters['prop_to_save'],stack=stack, section=section, dataType='false_typical',dataFolderName=dataFolderName)
        fun_save_matched_blobs_info(matched_paris=false_matched_paris, prop_to_save=scan_parameters['prop_to_save'],stack=stack, section=section, dataType='false_matched',dataFolderName=dataFolderName)
    print('Section %d result saved.'% section)
    return 0
# data_typical_blobs[section] = typical_blobs
# data_matched_paris[section] = matched_paris
# data_false_typical_blobs[section] = false_typical_blobs
# data_false_matched_pairs[section] = false_matched_paris   

In [9]:
from multiprocess import Pool

In [9]:
scan_parameters['dataFolderName'] = 'NoCoors_FST07'
scan_parameters['scan_outside_margin'] = 3000
scan_parameters['false_match_sample_num'] = 200

In [10]:
fun_collect_blobs(section=92, scan_parameters=scan_parameters, collectFlaseMatched=False, saveMatched=False)

Section 92 Finished percentage: 0.000000
Section 92 Finished percentage: 1.916847
Section 92 Finished percentage: 3.833694
Section 92 Finished percentage: 5.750542
Section 92 Finished percentage: 7.667389
Section 92 Finished percentage: 9.584236
Section 92 Finished percentage: 11.501083
Section 92 Finished percentage: 13.417930
Section 92 Finished percentage: 15.334777
Section 92 Finished percentage: 17.251625
Section 92 Finished percentage: 19.168472
Section 92 Finished percentage: 21.085319
Section 92 Finished percentage: 23.002166
Section 92 Finished percentage: 24.919013
Section 92 Finished percentage: 26.835860
Section 92 Finished percentage: 28.752708
Section 92 Finished percentage: 30.669555
Section 92 Finished percentage: 32.586402
Section 92 Finished percentage: 34.503249
Section 92 Finished percentage: 36.420096
Section 92 Finished percentage: 38.336943
Section 92 Finished percentage: 40.253791
Section 92 Finished percentage: 42.170638
Section 92 Finished percentage: 44.08748

0

In [ ]:
# To save the coords, only 4 process can be parallelized to avoid 
pool = Pool(4)
pool.map(lambda sec: fun_collect_blobs(section=sec, scan_parameters=scan_parameters, collectFlaseMatched=True, saveMatched=True), secList)

Section 92 Finished percentage: 0.000000
Section 92 Finished percentage: 1.916847
Section 92 Finished percentage: 3.833694
Section 92 Finished percentage: 5.750542
Section 92 Finished percentage: 7.667389
Section 92 Finished percentage: 9.584236
Section 92 Finished percentage: 11.501083
Section 92 Finished percentage: 13.417930
Section 92 Finished percentage: 15.334777
Section 92 Finished percentage: 17.251625
Section 129 Finished percentage: 0.000000
Section 92 Finished percentage: 19.168472
Section 92 Finished percentage: 21.085319
Section 92 Finished percentage: 23.002166
Section 129 Finished percentage: 1.817389
Section 110 Finished percentage: 0.000000
Section 92 Finished percentage: 24.919013
Section 92 Finished percentage: 26.835860
Section 129 Finished percentage: 3.634778
Section 110 Finished percentage: 1.859324
Section 92 Finished percentage: 28.752708
Section 92 Finished percentage: 30.669555
Section 129 Finished percentage: 5.452166
Section 92 Finished percentage: 32.58640

Section 147 Finished percentage: 53.342026
Section 110 Finished percentage: 96.684826
Section 147 Finished percentage: 54.823749
Section 129 Finished percentage: 94.504216
Section 110 Finished percentage: 98.544150
Section 147 Finished percentage: 56.305472
Section 129 Finished percentage: 96.321605
Section 147 Finished percentage: 57.787195
Section 129 Finished percentage: 98.138994
Section 147 Finished percentage: 59.268918
Section 129 Finished percentage: 99.956383
Section 147 Finished percentage: 60.750641
Section 147 Finished percentage: 62.232364
Section 147 Finished percentage: 63.714087
Section 147 Finished percentage: 65.195810
Section 147 Finished percentage: 66.677533
Section 147 Finished percentage: 68.159256
Section 147 Finished percentage: 69.640979
Section 147 Finished percentage: 71.122701
Section 147 Finished percentage: 72.604424
Section 147 Finished percentage: 74.086147
Section 147 Finished percentage: 75.567870
Section 147 Finished percentage: 77.049593
Section 147

Section 130 Finished percentage: 82.481029
Section 130 Finished percentage: 84.130650
Section 130 Finished percentage: 85.780271
Section 130 Finished percentage: 87.429891
Section 147 result saved.
Section 130 Finished percentage: 89.079512
Section 130 Finished percentage: 90.729132
Section 130 Finished percentage: 92.378753
Section 130 Finished percentage: 94.028373
Section 130 Finished percentage: 95.677994
Section 130 Finished percentage: 97.327615
Section 130 Finished percentage: 98.977235
Section 148 Finished percentage: 0.000000
Section 148 Finished percentage: 1.394156
Section 148 Finished percentage: 2.788311
Section 148 Finished percentage: 4.182467
Section 148 Finished percentage: 5.576623
Section 148 Finished percentage: 6.970778
Section 148 Finished percentage: 8.364934
Section 148 Finished percentage: 9.759090
Section 148 Finished percentage: 11.153246
Section 148 Finished percentage: 12.547401
Section 148 Finished percentage: 13.941557
Section 148 Finished percentage: 15.

Section 131 Finished percentage: 61.826820
Section 94 Finished percentage: 19.082149
Section 94 Finished percentage: 20.990364
Section 131 Finished percentage: 63.453841
Section 131 Finished percentage: 65.080863
Section 94 Finished percentage: 22.898578
Section 131 Finished percentage: 66.707885
Section 94 Finished percentage: 24.806793
Section 131 Finished percentage: 68.334906
Section 94 Finished percentage: 26.715008
Section 131 Finished percentage: 69.961928
Section 94 Finished percentage: 28.623223
Section 131 Finished percentage: 71.588949
Section 94 Finished percentage: 30.531438
Section 131 Finished percentage: 73.215971
Section 94 Finished percentage: 32.439653
Section 131 Finished percentage: 74.842992
Section 94 Finished percentage: 34.347868
Section 131 Finished percentage: 76.470014
Section 94 Finished percentage: 36.256082
Section 131 Finished percentage: 78.097036
Section 94 Finished percentage: 38.164297
Section 94 Finished percentage: 40.072512
Section 131 Finished pe

Section 113 Finished percentage: 97.952244
Section 149 Finished percentage: 64.640246
Section 113 Finished percentage: 99.800399
Section 132 Finished percentage: 47.528517
Section 149 Finished percentage: 66.179300
Section 132 Finished percentage: 49.167431
Section 149 Finished percentage: 67.718353
Section 132 Finished percentage: 50.806346
Section 149 Finished percentage: 69.257407
Section 132 Finished percentage: 52.445260
Section 149 Finished percentage: 70.796460
Section 132 Finished percentage: 54.084175
Section 149 Finished percentage: 72.335514
Section 149 Finished percentage: 73.874567
Section 132 Finished percentage: 55.723089
Section 149 Finished percentage: 75.413621
Section 132 Finished percentage: 57.362003
Section 149 Finished percentage: 76.952674
Section 132 Finished percentage: 59.000918
Section 149 Finished percentage: 78.491728
Section 132 Finished percentage: 60.639832
Section 149 Finished percentage: 80.030781
Section 132 Finished percentage: 62.278747
Section 149

...

Section 101 Finished percentage: 11.405977
Section 101 Finished percentage: 13.306973
Section 101 Finished percentage: 15.207969
Section 101 Finished percentage: 17.108965
Section 101 Finished percentage: 19.009961
Section 101 Finished percentage: 20.910957
Section 101 Finished percentage: 22.811953
Section 101 Finished percentage: 24.712950
Section 101 Finished percentage: 26.613946
Section 101 Finished percentage: 28.514942
Section 101 Finished percentage: 30.415938
Section 101 Finished percentage: 32.316934
Section 101 Finished percentage: 34.217930
Section 101 Finished percentage: 36.118926
Section 101 Finished percentage: 38.019922
Section 101 Finished percentage: 39.920919
Section 101 Finished percentage: 41.821915
Section 101 Finished percentage: 43.722911
Section 101 Finished percentage: 45.623907
Section 101 Finished percentage: 47.524903
Section 101 Finished percentage: 49.425899
Section 101 Finished percentage: 51.326895
Section 101 Finished percentage: 53.227891
Section 101

Section 126 Finished percentage: 56.117772
Section 157 Finished percentage: 75.187970
Section 126 Finished percentage: 57.988365
Section 157 Finished percentage: 76.530612
Section 157 Finished percentage: 77.873255
Section 126 Finished percentage: 59.858957
Section 126 Finished percentage: 61.729550
Section 157 Finished percentage: 79.215897
Section 126 Finished percentage: 63.600142
Section 157 Finished percentage: 80.558539
Section 126 Finished percentage: 65.470735
Section 157 Finished percentage: 81.901182
Section 126 Finished percentage: 67.341327
Section 126 Finished percentage: 69.211919
Section 157 Finished percentage: 83.243824
Section 126 Finished percentage: 71.082512
Section 157 Finished percentage: 84.586466
Section 126 Finished percentage: 72.953104
Section 126 Finished percentage: 74.823697
Section 126 Finished percentage: 76.694289
Section 157 Finished percentage: 85.929108
Section 126 Finished percentage: 78.564881
Section 157 Finished percentage: 87.271751
Section 126

Section 102 Finished percentage: 58.854904
Section 158 Finished percentage: 27.604242
Section 127 Finished percentage: 65.559828
Section 102 Finished percentage: 60.694120
Section 158 Finished percentage: 29.057097
Section 127 Finished percentage: 67.331715
Section 102 Finished percentage: 62.533336
Section 158 Finished percentage: 30.509952
Section 127 Finished percentage: 69.103602
Section 158 Finished percentage: 31.962807
Section 102 Finished percentage: 64.372552
Section 127 Finished percentage: 70.875489
Section 158 Finished percentage: 33.415662
Section 127 Finished percentage: 72.647377
Section 102 Finished percentage: 66.211767
Section 127 Finished percentage: 74.419264
Section 102 Finished percentage: 68.050983
Section 158 Finished percentage: 34.868517
Section 127 Finished percentage: 76.191151
Section 102 Finished percentage: 69.890199
Section 158 Finished percentage: 36.321371
Section 127 Finished percentage: 77.963038
Section 102 Finished percentage: 71.729415
Section 127

Section 128 Finished percentage: 75.903544
Section 128 Finished percentage: 77.849789
Section 128 Finished percentage: 79.796034
Section 128 Finished percentage: 81.742278
Section 128 Finished percentage: 83.688523
Section 128 Finished percentage: 85.634768
Section 128 Finished percentage: 87.581012
Section 128 Finished percentage: 89.527257
Section 128 Finished percentage: 91.473502
Section 128 Finished percentage: 93.419747
Section 128 Finished percentage: 95.365991
Section 128 Finished percentage: 97.312236
Section 128 Finished percentage: 99.258481
Section 159 Finished percentage: 0.000000
Section 159 Finished percentage: 1.352119
Section 159 Finished percentage: 2.704238
Section 159 Finished percentage: 4.056356
Section 159 Finished percentage: 5.408475
Section 159 Finished percentage: 6.760594
Section 159 Finished percentage: 8.112713
Section 159 Finished percentage: 9.464831
Section 159 Finished percentage: 10.816950
Section 159 Finished percentage: 12.169069
Section 159 Finishe

Section 143 Finished percentage: 55.200136
Section 103 Finished percentage: 98.128159
Section 165 Finished percentage: 12.852720
Section 143 Finished percentage: 56.615524
Section 143 Finished percentage: 58.030912
Section 103 Finished percentage: 99.979634
Section 165 Finished percentage: 14.021149
Section 143 Finished percentage: 59.446300
Section 165 Finished percentage: 15.189578
Section 143 Finished percentage: 60.861688
Section 165 Finished percentage: 16.358007
Section 143 Finished percentage: 62.277076
Section 165 Finished percentage: 17.526436
Section 143 Finished percentage: 63.692464
Section 143 Finished percentage: 65.107853
Section 165 Finished percentage: 18.694865
Section 143 Finished percentage: 66.523241
Section 165 Finished percentage: 19.863294
Section 143 Finished percentage: 67.938629
Section 165 Finished percentage: 21.031723
Section 143 Finished percentage: 69.354017
Section 165 Finished percentage: 22.200152
Section 143 Finished percentage: 70.769405
Section 143

In [ ]:
upload_to_s3('/shared/blob_matching_atlas',is_dir=True)

# For debuging

In [13]:
scan_range = scan_parameters['scan_section_range']
im0max = scan_parameters['im0max']
im1max = scan_parameters['im1max']
prop = scan_parameters['prop']
prop_for_comparison = scan_parameters['prop_for_comparison']
compare_weight = scan_parameters['compare_weight']
o_simil_threshold = scan_parameters['similarity_threshold']
o_fix_scan_size = scan_parameters['o_fix_scan_size']
o_scan_size_coeff = scan_parameters['scan_size_coeff']
o_scan_size = scan_parameters['scan_size']  
secList = scan_parameters['secList'] 
stack = scan_parameters['stack']

data_typical_blobs = {}
data_matched_paris = {}
data_false_typical_blobs = {}
data_false_matched_pairs = {}
data_blob_prop_dic = {}
data_blob_idx_selection = {}
section = 154
# for section in secList:
typical_blobs = []
matched_paris = []
false_typical_blobs = []
false_matched_paris = []
cell_centroids = {}
cell_numbers = {}
cell_global_coord = {}
im_blob_prop = {}
im_label_ori = {}
im_label = {}
im_BW = {}
sec_load_data_list = range(section - scan_range, section + scan_range + 1)
scan_section = list(sec_load_data_list)
scan_section.remove(section)
### loading data and reconstructed labeled images ###
tempList = list(sec_load_data_list)
for tempSec in tempList:
    if tempSec in secList:
        cell_global_coord[tempSec] = load_cell_data('coords', stack=stack, sec=tempSec)
        temp_im_label, temp_im_blob_prop, _ = fun_reconstruct_labeled_image(cell_global_coord[tempSec],crop_range= scan_parameters['crop_range_mxmx'], 
                                                                    oriImL0=scan_parameters['oriImL0'],oriImL1=scan_parameters['oriImL1'])
        im_label[tempSec] = temp_im_label
        im_BW[tempSec] = temp_im_label > 0
        im_blob_prop[tempSec] = np.array(temp_im_blob_prop)
    else:
        sys.stderr.write('Warning: missing section %d'%tempSec)
        scan_section.remove(tempSec)
        sec_load_data_list.remove(tempSec)

### Start getting region properties of each blob    ###
blob_prop_dic = fun_regionprops_dic(im_blob_prop=im_blob_prop,scan_parameters=scan_parameters)
data_blob_prop_dic[section] = blob_prop_dic[section] # Save data
blob_idx_selection = {tempSec : np.logical_and.reduce(np.row_stack(
    (
#      blob_prop_dic[tempSec]['compactness']<2.5,
     blob_prop_dic[tempSec]['area']<3000,
     blob_prop_dic[tempSec]['area']>200))) for tempSec in sec_load_data_list}  

data_blob_idx_selection[section] = blob_idx_selection[section] # Save data

n_blobs = {tempSec: len(im_blob_prop[tempSec]) for tempSec in im_blob_prop.keys()}
secList_in_BlobPropDic = im_blob_prop.keys()
if set(scan_section).issubset(set(secList_in_BlobPropDic)):
    pass
else:
    print('Warrning: Scaned section(s) not included in input im_blob_prop')



### Start scanning ###
for blobID in range(n_blobs[section]):
    if (blobID % 1000 == 0):
        print('Section %d Finished percentage: %f'%(section, (float(blobID)*100 / n_blobs[section]) )) 

    temp_curr_blob_props = {}
    for tempProp in prop:
        temp_curr_blob_props[tempProp] = blob_prop_dic[section][tempProp][blobID]
    if not blob_idx_selection[section][blobID]:
#         print('Blob %d eliminated'%blobID)
        continue

    tempB1_idx_loc = temp_curr_blob_props['centroid']
    if o_fix_scan_size:
        temp_next_sec_range, local_cloc = fun_scan_range(tempB1_idx_loc,o_scan_size,im0max=im0max,im1max=im1max,o_form='2D')
        temp_next_sec_range_1D,_ = fun_scan_range(tempB1_idx_loc,o_scan_size,im0max=im0max,im1max=im1max)
    else:
        temp_next_sec_range, local_cloc = fun_scan_range(tempB1_idx_loc,o_scan_size_coeff*fun_radius_bbox(*temp_curr_blob_props.bbox),im0max=im0max,im1max=im1max,o_form='2D')
        temp_next_sec_range_1D,_ = fun_scan_range(tempB1_idx_loc,o_scan_size_coeff*fun_radius_bbox(*temp_curr_blob_props.bbox),im0max=im0max,im1max=im1max)


    for tempSec in scan_section:
#         print('Blbo %d left. Start scanning'%blobID)
        if tempSec not in secList_in_BlobPropDic:
            continue

        # Find blobs at the nearby location in the scaned section
        tempBlobInside = fun_blobs_in_polygen(blob_prop_dic[tempSec]['centroid'],temp_next_sec_range,coor_order='cr')
        tempBlobOutside = fun_blobs_out_polygen(blob_prop_dic[tempSec]['centroid'], temp_next_sec_range, coor_order='cr', margin=scan_parameters['scan_outside_margin'])
        tempBlobInsideIndex = np.where(tempBlobInside)[0]
        temp_num_blob = len(tempBlobInsideIndex)
        tempBlobOutsideIndex = np.where(tempBlobOutside)[0]
        tempBlobOutsideIndex = np.random.choice(tempBlobOutsideIndex, size=scan_parameters['false_match_sample_num'], replace=False)

        temp_num_outblob = len(tempBlobOutsideIndex)

        ### Typical blobs###

        if temp_num_blob:
            temp_sim = {}
            for temp_prop in prop_for_comparison:
                      temp_sim[temp_prop] = np.array(fun_similarity(temp_curr_blob_props[temp_prop], blob_prop_dic[tempSec][temp_prop][tempBlobInside],distance_type=temp_prop))
            temp_sim_matrix = np.column_stack((temp_sim[temp_prop] for temp_prop in prop_for_comparison))

            #### Blob comparison ####
            temp_weighted_sim = np.dot(temp_sim_matrix,compare_weight)
            temp_compare_result = temp_weighted_sim > o_simil_threshold
            if any(temp_compare_result.tolist()):
                typical_blobs.append([section,blobID, im_blob_prop[section][blobID]])
                matched_paris.append([section,blobID,
                                      im_blob_prop[section][blobID],
                                      tempSec,
                                      tempBlobInsideIndex[temp_compare_result],
                                      im_blob_prop[tempSec][tempBlobInsideIndex[temp_compare_result]],
                                      temp_sim_matrix[temp_compare_result,:]])
        else:
            pass

        ### False-typical blobs###
        if temp_num_outblob:
            temp_false_sim = {}
            for temp_prop in prop_for_comparison:
                      temp_false_sim[temp_prop] = np.array(fun_similarity(temp_curr_blob_props[temp_prop], blob_prop_dic[tempSec][temp_prop][tempBlobOutsideIndex],distance_type=temp_prop))
            temp_false_sim_matrix = np.column_stack((temp_false_sim[temp_prop] for temp_prop in prop_for_comparison))

            #### Blob comparison ####
            temp_weighted_false_sim = np.dot(temp_false_sim_matrix,compare_weight)
            temp_false_compare_result = temp_weighted_false_sim > o_simil_threshold
            if any(temp_false_compare_result.tolist()):
                false_typical_blobs.append([section,blobID, im_blob_prop[section][blobID]])
                false_matched_paris.append([section,blobID,im_blob_prop[section][blobID],
                                            tempSec, 
                                            tempBlobOutsideIndex[temp_false_compare_result], 
                                            im_blob_prop[tempSec][tempBlobOutsideIndex[temp_false_compare_result]],
                                            temp_false_sim_matrix[temp_false_compare_result,:]])
        else:
            pass
fun_save_typical_blobs_info(typical_blobs=typical_blobs, prop_to_save=scan_parameters['prop_to_save'],stack=stack, sec=section, dataType='typical')
fun_save_matched_blobs_info(matched_paris=matched_paris, prop_to_save=scan_parameters['prop_to_save'],stack=stack, sec=section, dataType='matched')
fun_save_typical_blobs_info(typical_blobs=false_typical_blobs, prop_to_save=scan_parameters['prop_to_save'],stack=stack, sec=section, dataType='false_typical')
fun_save_matched_blobs_info(matched_paris=false_matched_paris, prop_to_save=scan_parameters['prop_to_save'],stack=stack, sec=section, dataType='false_matched')

# data_typical_blobs[section] = typical_blobs
# data_matched_paris[section] = matched_paris
# data_false_typical_blobs[section] = false_typical_blobs
# data_false_matched_pairs[section] = false_matched_paris   

Section 154 Finished percentage: 0.000000
Section 154 Finished percentage: 1.460814
Section 154 Finished percentage: 2.921627
Section 154 Finished percentage: 4.382441
Section 154 Finished percentage: 5.843255
Section 154 Finished percentage: 7.304068
Section 154 Finished percentage: 8.764882
Section 154 Finished percentage: 10.225696
Section 154 Finished percentage: 11.686509
Section 154 Finished percentage: 13.147323
Section 154 Finished percentage: 14.608137
Section 154 Finished percentage: 16.068950
Section 154 Finished percentage: 17.529764
Section 154 Finished percentage: 18.990578
Section 154 Finished percentage: 20.451391
Section 154 Finished percentage: 21.912205
Section 154 Finished percentage: 23.373019
Section 154 Finished percentage: 24.833832
Section 154 Finished percentage: 26.294646
Section 154 Finished percentage: 27.755460
Section 154 Finished percentage: 29.216273
Section 154 Finished percentage: 30.677087
Section 154 Finished percentage: 32.137901
Section 154 Finish

In [30]:
# def fun_collect_blobs(section, scan_parameters, collectFlaseMatched=False, saveMatched=True):
section = 92
collectFlaseMatched=False
saveMatched=True
scan_range = scan_parameters['scan_section_range']
im0max = scan_parameters['im0max']
im1max = scan_parameters['im1max']
prop = scan_parameters['prop']
prop_for_comparison = scan_parameters['prop_for_comparison']
compare_weight = scan_parameters['compare_weight']
o_simil_threshold = scan_parameters['similarity_threshold']
o_false_simil_threshold = scan_parameters['false_similarity_threshold'] # Option. Default is equal to o_simil_threshold
o_fix_scan_size = scan_parameters['o_fix_scan_size']
o_scan_size_coeff = scan_parameters['scan_size_coeff']
o_scan_size = scan_parameters['scan_size']  
secList = scan_parameters['secList'] 
stack = scan_parameters['stack']
dataFolderName = scan_parameters['dataFolderName']
# data_typical_blobs = {}
# data_matched_paris = {}
# data_false_typical_blobs = {}
# data_false_matched_pairs = {}
# data_blob_prop_dic = {}
# data_blob_idx_selection = {}
#     section = 154
# for section in secList:
typical_blobs = []
matched_paris = []
false_typical_blobs = []
false_matched_paris = []
cell_centroids = {}
cell_numbers = {}
cell_global_coord = {}
im_blob_prop = {}
im_label = {}
im_BW = {}
sec_load_data_list = range(section - scan_range, section + scan_range + 1)
scan_section = list(sec_load_data_list)
scan_section.remove(section)
### loading data and reconstructed labeled images ###
tempList = list(sec_load_data_list)
for tempSec in tempList:
    if tempSec in secList:
        cell_global_coord[tempSec] = load_cell_data('coords', stack=stack, sec=tempSec)
        _, temp_im_blob_prop, _ = fun_reconstruct_labeled_image(cell_global_coord[tempSec],crop_range= scan_parameters['crop_range_mxmx'], 
                                                                    oriImL0=scan_parameters['oriImL0'],oriImL1=scan_parameters['oriImL1'])
#             im_label[tempSec] = temp_im_label
#             im_BW[tempSec] = temp_im_label > 0
        im_blob_prop[tempSec] = np.array(temp_im_blob_prop)
    else:
        sys.stderr.write('Warning: missing section %d'%tempSec)
        scan_section.remove(tempSec)
        sec_load_data_list.remove(tempSec)

### Start getting region properties of each blob    ###
blob_prop_dic = fun_regionprops_dic(im_blob_prop=im_blob_prop,scan_parameters=scan_parameters)
# data_blob_prop_dic[section] = blob_prop_dic[section] # Save data
blob_idx_selection = {tempSec : np.logical_and.reduce(np.row_stack(
    (
    #      blob_prop_dic[tempSec]['compactness']<2.5,
     blob_prop_dic[tempSec]['area']<3000,
     blob_prop_dic[tempSec]['area']>200))) for tempSec in sec_load_data_list}  

# data_blob_idx_selection[section] = blob_idx_selection[section] # Save data

n_blobs = {tempSec: len(im_blob_prop[tempSec]) for tempSec in im_blob_prop.keys()}
secList_in_BlobPropDic = im_blob_prop.keys()
if set(scan_section).issubset(set(secList_in_BlobPropDic)):
    pass
else:
    print('Warrning: Scaned section(s) not included in input im_blob_prop')



### Start scanning ###
for blobID in range(n_blobs[section]):
    if (blobID % 1000 == 0):
        print('Section %d Finished percentage: %f'%(section, (float(blobID)*100 / n_blobs[section]) ))

    temp_curr_blob_props = {}
    for tempProp in prop:
        temp_curr_blob_props[tempProp] = blob_prop_dic[section][tempProp][blobID]
    if not blob_idx_selection[section][blobID]:
#         print('Blob %d eliminated'%blobID)
        continue

    tempB1_idx_loc = temp_curr_blob_props['centroid']
    if o_fix_scan_size:
        temp_next_sec_range, local_cloc = fun_scan_range(tempB1_idx_loc,o_scan_size,im0max=im0max,im1max=im1max,o_form='2D')
        temp_next_sec_range_1D,_ = fun_scan_range(tempB1_idx_loc,o_scan_size,im0max=im0max,im1max=im1max)
    else:
        temp_next_sec_range, local_cloc = fun_scan_range(tempB1_idx_loc,o_scan_size_coeff*fun_radius_bbox(*temp_curr_blob_props.bbox),im0max=im0max,im1max=im1max,o_form='2D')
        temp_next_sec_range_1D,_ = fun_scan_range(tempB1_idx_loc,o_scan_size_coeff*fun_radius_bbox(*temp_curr_blob_props.bbox),im0max=im0max,im1max=im1max)


    for tempSec in scan_section:
#         print('Blbo %d left. Start scanning'%blobID)
        if tempSec not in secList_in_BlobPropDic:
            continue

        # Find blobs at the nearby location in the scaned section
        tempBlobInside = fun_blobs_in_polygen(blob_prop_dic[tempSec]['centroid'],temp_next_sec_range,coor_order='cr')
        tempBlobInsideIndex = np.where(tempBlobInside)[0]
        temp_num_blob = len(tempBlobInsideIndex)
        if collectFlaseMatched == True:
            tempBlobOutside = fun_blobs_out_polygen(blob_prop_dic[tempSec]['centroid'], temp_next_sec_range, coor_order='cr', margin=scan_parameters['scan_outside_margin'])
            tempBlobOutsideIndex = np.where(tempBlobOutside)[0]
            tempBlobOutsideIndex = np.random.choice(tempBlobOutsideIndex, size=scan_parameters['false_match_sample_num'], replace=False)
            temp_num_outblob = len(tempBlobOutsideIndex)

        ### Typical blobs###

        if temp_num_blob:
            temp_sim = {}
            for temp_prop in prop_for_comparison:
                      temp_sim[temp_prop] = np.array(fun_similarity(temp_curr_blob_props[temp_prop], blob_prop_dic[tempSec][temp_prop][tempBlobInside],distance_type=temp_prop))
            temp_sim_matrix = np.column_stack((temp_sim[temp_prop] for temp_prop in prop_for_comparison))

            #### Blob comparison ####
            temp_weighted_sim = np.dot(temp_sim_matrix,compare_weight)
            temp_compare_result = temp_weighted_sim > o_simil_threshold
            if any(temp_compare_result.tolist()):
                typical_blobs.append([section,blobID, im_blob_prop[section][blobID]])
                matched_paris.append([section,blobID,
                                      im_blob_prop[section][blobID],
                                      tempSec,
                                      tempBlobInsideIndex[temp_compare_result],
                                      im_blob_prop[tempSec][tempBlobInsideIndex[temp_compare_result]],
                                      temp_sim_matrix[temp_compare_result,:]])
        else:
            pass

        ### False-typical blobs###
        if collectFlaseMatched == True:
            if temp_num_outblob:
                temp_false_sim = {}
                for temp_prop in prop_for_comparison:
                          temp_false_sim[temp_prop] = np.array(fun_similarity(temp_curr_blob_props[temp_prop], blob_prop_dic[tempSec][temp_prop][tempBlobOutsideIndex],distance_type=temp_prop))
                temp_false_sim_matrix = np.column_stack((temp_false_sim[temp_prop] for temp_prop in prop_for_comparison))

                #### Blob comparison ####
                temp_weighted_false_sim = np.dot(temp_false_sim_matrix,compare_weight)
                temp_false_compare_result = temp_weighted_false_sim > o_false_simil_threshold
                if any(temp_false_compare_result.tolist()):
                    false_typical_blobs.append([section,blobID, im_blob_prop[section][blobID]])
                    false_matched_paris.append([section,blobID,im_blob_prop[section][blobID],
                                                tempSec, 
                                                tempBlobOutsideIndex[temp_false_compare_result], 
                                                im_blob_prop[tempSec][tempBlobOutsideIndex[temp_false_compare_result]],
                                                temp_false_sim_matrix[temp_false_compare_result,:]])
            else:
                pass
tempFp = get_typical_cell_data_filepath(what='scan_parameters',stack=stack,sec=section,dataFolderName=dataFolderName)
create_if_not_exists(os.path.dirname(tempFp))
save_pickle(scan_parameters,tempFp)
fun_save_typical_blobs_info(typical_blobs=typical_blobs, prop_to_save=scan_parameters['prop_to_save'],stack=stack, section=section, dataType='typical',dataFolderName=dataFolderName)
if saveMatched == True:
    fun_save_matched_blobs_info(matched_paris=matched_paris, prop_to_save=scan_parameters['prop_to_save'],stack=stack, section=section, dataType='matched',dataFolderName=dataFolderName)
if collectFlaseMatched == True:
    fun_save_typical_blobs_info(typical_blobs=false_typical_blobs, prop_to_save=scan_parameters['prop_to_save'],stack=stack, section=section, dataType='false_typical',dataFolderName=dataFolderName)
    fun_save_matched_blobs_info(matched_paris=false_matched_paris, prop_to_save=scan_parameters['prop_to_save'],stack=stack, section=section, dataType='false_matched',dataFolderName=dataFolderName)
print('Section %d result saved.'% section)
# return 0
# data_typical_blobs[section] = typical_blobs
# data_matched_paris[section] = matched_paris
# data_false_typical_blobs[section] = false_typical_blobs
# data_false_matched_pairs[section] = false_matched_paris   

Section 92 Finished percentage: 0.000000
Section 92 Finished percentage: 1.916847
Section 92 Finished percentage: 3.833694
Section 92 Finished percentage: 5.750542
Section 92 Finished percentage: 7.667389
Section 92 Finished percentage: 9.584236
Section 92 Finished percentage: 11.501083
Section 92 Finished percentage: 13.417930
Section 92 Finished percentage: 15.334777
Section 92 Finished percentage: 17.251625
Section 92 Finished percentage: 19.168472
Section 92 Finished percentage: 21.085319
Section 92 Finished percentage: 23.002166
Section 92 Finished percentage: 24.919013
Section 92 Finished percentage: 26.835860
Section 92 Finished percentage: 28.752708
Section 92 Finished percentage: 30.669555
Section 92 Finished percentage: 32.586402
Section 92 Finished percentage: 34.503249
Section 92 Finished percentage: 36.420096
Section 92 Finished percentage: 38.336943
Section 92 Finished percentage: 40.253791
Section 92 Finished percentage: 42.170638
Section 92 Finished percentage: 44.08748

TypeError: cannot concatenate 'str' and 'NoneType' objects

In [34]:
scan_parameters['prop_to_save']

['moments_hu',
 'centroid',
 'area',
 'eccentricity',
 'equivalent_diameter',
 'compactness',
 'orientation',
 'solidity']

In [37]:
fun_save_typical_blobs_info(typical_blobs=typical_blobs, prop_to_save=scan_parameters['prop_to_save'],stack=stack, section=section, dataType='typical',dataFolderName=dataFolderName)

KeyError: 'compactness'